In [1]:
import numpy as np
import math
import pandas 
import random 
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [2]:
dataframe = pandas.read_csv("ionosphere_data.csv")

In [3]:
#shuffle
dataset = np.array(dataframe.values)
dataset_shuf = []
index_shuf = list(range(len(dataset)))
random.shuffle(index_shuf)
for i in index_shuf:
    dataset_shuf.append(dataset[i,:])   
dataset_shuf = np.array(dataset_shuf)
dataset_shuf.reshape(350,35)

print("")

In [4]:
#ont hot g = 0 b = 1
for i in range(350) :
    if dataset_shuf[i,34] == 'g' :
        dataset_shuf[i,34] = 0
    elif dataset_shuf[i,34] == 'b' :
        dataset_shuf[i,34] = 1
#print(dataset_shuf)

print("")

In [5]:
training_x = dataset_shuf[:270,0:34]
training_y = dataset_shuf[:270,34:35]
training_x.reshape(270,34)
training_y.reshape(270,1)

testing_x = dataset_shuf[270:,0:34]
testing_y = dataset_shuf[270:,34:35]
testing_x.reshape(80,34)
testing_y.reshape(80,1)
print("")

In [8]:
def relu(x) :
    return x * (x > 0) + 0.01 * x * (x <= 0)

def relu_der(x) :
    return 1 * (x > 0) + 0.01 * (x <= 0)
    
def sigmoid(x) :
    x = x.astype('float64')
    return 1/(1+np.exp(-x))

def sigmoid_der(x) :
    x = x.astype('float64')
    return sigmoid(x) * (1 - sigmoid(x))

def error(x, y) :
    CEE = 0
    for i in range(len(y)):
        if y[i] == 0 :
            CEE += -math.log(1-x[i])
        elif y[i] == 1:    
            CEE += -math.log(x[i])
    return CEE

In [ ]:
epochtimes = 200000
mini_batch = 16

    
for i in range(2):
    
    preloss = 1e30   #decay the learning rate
    lastLayerNode = i + 2

    #define weight:

    wh = np.random.random((34,34))-0.5
    wh2 = np.random.random((34,68))-0.5
    wh3 = np.random.random((68,17))-0.5
    wh4 = np.random.random((17,8))-0.5
    wh5 = np.random.random((8,lastLayerNode))-0.5
    wo = np.random.random((lastLayerNode,1))-0.5
    lr = 0.003

    bias = 0.02
    bias1 = 0.02
    bias2 = 0.02
    bias3 = 0.02
    bias4 = 0.02
    bias5 = 0.02

    ######
    
    lossCurve = []
    
    for epoch in range(epochtimes):
        
        #minibatch
        training_x_random = []
        training_y_random = []
        for i in range(mini_batch):
            rindex = random.randint(0,len(training_x)-1)
            training_x_random.append(training_x[rindex,:])
            training_y_random.append(training_y[rindex,:])
        training_x_random = np.array(training_x_random)
        training_y_random = np.array(training_y_random)



        ih = np.dot(training_x_random, wh) + bias
        oh = relu(ih)

        ih2 = np.dot(oh, wh2) + bias1
        oh2 = relu(ih2)
        ih3 = np.dot(oh2, wh3) + bias2
        oh3 = relu(ih3)
        ih4 = np.dot(oh3, wh4) + bias3
        oh4 = relu(ih4)
        ih5 = np.dot(oh4, wh5) + bias4
        oh5 = relu(ih5)
        input_op = np.dot(oh5, wo)+ bias5
        output_op = sigmoid(input_op)


        #1

        de_doo = output_op - training_y_random
        doo_dio = sigmoid_der(input_op)
        dino_dwo = oh5      
        de_dwo = np.dot(dino_dwo.T, de_doo * doo_dio)
        de_db5 = de_doo * doo_dio

        #2

        de_dio = de_doo * doo_dio
        dio_doh5 = wo
        de_doh5 = np.dot(de_dio, dio_doh5.T)
        doh5_dih5 = relu_der(ih5)
        dih5_dwh5 = oh4
        de_dwh5 = np.dot(dih5_dwh5.T, doh5_dih5 * de_doh5)
        de_db4 = doh5_dih5 * de_doh5

        #3

        de_dih5 = de_doh5 * doh5_dih5
        dih5_doh4 = wh5
        de_doh4 = np.dot(de_dih5, dih5_doh4.T)
        doh4_dih4 = relu_der(ih4)
        dinh4_dwh4 = oh3
        de_dwh4 = np.dot(dinh4_dwh4.T, doh4_dih4 * de_doh4)
        de_db3 = doh4_dih4 * de_doh4

        #4

        de_dih4 = de_doh4 * doh4_dih4
        dih4_doh3 = wh4
        de_doh3 = np.dot(de_dih4, dih4_doh3.T)
        doh3_dih3 = relu_der(ih3)
        dinh3_dwh3 = oh2
        de_dwh3 = np.dot(dinh3_dwh3.T, doh3_dih3 * de_doh3)
        de_db2 = doh3_dih3 * de_doh3

        #5

        de_dih3 = de_doh3 * doh3_dih3
        dih3_doh2 = wh3
        de_doh2 = np.dot(de_dih3, dih3_doh2.T)
        doh2_dih2 = relu_der(ih2)
        dinh2_dwh2 = oh
        de_dwh2 = np.dot(dinh2_dwh2.T, doh2_dih2 * de_doh2)
        de_db1 = doh2_dih2 * de_doh2

        #6

        de_dih2 = de_doh2 * doh2_dih2
        dinh2_douth = wh2
        de_doh = np.dot(de_dih2, dinh2_douth.T)
        douth_dinh = relu_der(ih)
        dinh_dwh = training_x_random
        de_dwh = np.dot(dinh_dwh.T, douth_dinh * de_doh)    
        de_db = douth_dinh * de_doh

        #==========================================



        wh = wh-lr * de_dwh
        wh2 = wh2-lr * de_dwh2
        wh3 = wh3-lr * de_dwh3
        wh4 = wh4-lr * de_dwh4
        wh5 = wh5-lr * de_dwh5
        wo = wo-lr * de_dwo

        bias -= lr * de_db.sum()/np.size(de_db)
        bias1 -= lr * de_db1.sum()/np.size(de_db1)
        bias2 -= lr * de_db2.sum()/np.size(de_db2)
        bias3 -= lr * de_db3.sum()/np.size(de_db3)
        bias4 -= lr * de_db4.sum()/np.size(de_db4)
        bias5 -= lr * de_db5.sum()/np.size(de_db5)


        #calulate error 
        if epoch % 1000 == 0 : 
            ih = np.dot(training_x, wh) + bias
            oh = relu(ih)
            ih2 = np.dot(oh, wh2) + bias1
            oh2 = relu(ih2)
            ih3 = np.dot(oh2, wh3) + bias2
            oh3 = relu(ih3)
            ih4 = np.dot(oh3, wh4) + bias3
            oh4 = relu(ih4)
            ih5 = np.dot(oh4, wh5) + bias4
            oh5 = relu(ih5)
            input_op = np.dot(oh5, wo)+ bias5
            output_op = sigmoid(input_op)
            error_out = error(output_op, training_y)
            lossCurve.append(error_out)

            #set decay for learning rate
            if error_out > preloss :
                lr *= 0.95

            preloss = error_out


            print('now training ',epoch*100/epochtimes,'%')
            #print(error_out)
            #print(error_out)
            #print('\n',lr * de_dwh,'\n', lr * de_dwo,'\n')


            #print(derror_wh, derror_wo)

            
    single_point = np.array(training_x)
    result1 = np.dot(single_point, wh)
    result2 = relu(result1)
    result3 = np.dot(result2, wh2)
    result4 = relu(result3)
    result5 = np.dot(result4, wh3)
    result6 = relu(result5)
    result7 = np.dot(result6, wh4)
    result8 = relu(result7)
    result9 = np.dot(result8, wh5)
    result10 = relu(result9)
    result11 = np.dot(result10, wo)
    result12 = sigmoid(result11)
    print("------------------------------------")
    print("last Layer Node: ",lastLayerNode)
    print("-")
    print("train error:",error(result12, training_y))
    
    single_point = np.array(testing_x)
    result1 = np.dot(single_point, wh)
    result2 = relu(result1)
    result3 = np.dot(result2, wh2)
    result4 = relu(result3)
    result5 = np.dot(result4, wh3)
    result6 = relu(result5)
    result7 = np.dot(result6, wh4)
    result8 = relu(result7)
    result9 = np.dot(result8, wh5)
    result10 = relu(result9)
    result11 = np.dot(result10, wo)
    result12 = sigmoid(result11)
    
    error_out = error(result12, testing_y)
    print("test error:",error_out)
    plt.plot(lossCurve)
    plt.title('learningCurve_loss')
    plt.xlabel('kilo epochs')

    plt.show()
    if (lastLayerNode == 2):
        
        group11 = [[],[]]
        group00 = [[],[]]
        for i in range(len(testing_x)):
            if(result12[i]>=0.5):
                for indexx in range(2):
                    group11[indexx].append(result10[i][indexx])
            else:
                for indexx in range(2):
                    group00[indexx].append(result10[i][indexx])
        fig, aax = plt.subplots()
        aax.scatter(group00[0], group00[1],c='b')
        aax.scatter(group11[0], group11[1],c='r')
        aax.set_title('2D feature '+str(epochtimes)+' epoch');
        plt.show()
    elif (lastLayerNode == 3):
        
        group1 = [[],[],[]]
        group0 = [[],[],[]]

        for i in range(len(testing_x)):
            if(result12[i]>=0.5):
                for indexx in range(3):
                    group1[indexx].append(result10[i][indexx])
            else:
                for indexx in range(3):
                    group0[indexx].append(result10[i][indexx])

        ax = plt.axes(projection='3d')
        ax.scatter3D(group1[0], group1[1], group1[2], c='r');
        ax.scatter3D(group0[0], group0[1], group0[2], c='b');
        ax.set_title('3D feature '+str(epochtimes)+' epoch');
        plt.show()
        print("")
    else:
        print("somethingWrong")
    print("------------------------------------")
print("end")

now training  0.0 %
now training  0.5 %
now training  1.0 %
now training  1.5 %
now training  2.0 %
now training  2.5 %
now training  3.0 %
now training  3.5 %
now training  4.0 %
now training  4.5 %
now training  5.0 %
now training  5.5 %
now training  6.0 %
now training  6.5 %
now training  7.0 %
now training  7.5 %
now training  8.0 %
now training  8.5 %
now training  9.0 %
now training  9.5 %
now training  10.0 %
now training  10.5 %
now training  11.0 %
now training  11.5 %
now training  12.0 %
now training  12.5 %
now training  13.0 %
now training  13.5 %
now training  14.0 %
now training  14.5 %
now training  15.0 %
now training  15.5 %
now training  16.0 %
now training  16.5 %
now training  17.0 %
now training  17.5 %
now training  18.0 %
now training  18.5 %
now training  19.0 %
now training  19.5 %
now training  20.0 %
now training  20.5 %
now training  21.0 %
now training  21.5 %
